In [2]:
import anvil.server

anvil.server.connect("WQ3UBRHS4OF3TJLPRMBMHWSA-4MAZVW7HBP3WMJIX")

In [3]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


import os
import anvil.media
import cv2
from collections import Counter
from pprint import pprint as pp
import numpy as np

location_image =  'C:/Users/geral/Documents/SeniorProject/Predict'

@anvil.server.callable

def predict_image(imageA):
    """Predict."""
    # [START automl_vision_object_detection_predict]
    from google.cloud import automl
    
    with anvil.media.TempFile(imageA) as FN1:
        #file_path = FN1
        imgA = cv2.imread(FN1)
        
        height = imgA.shape[0]
        width = imgA.shape[1]
        
        print(height, width)
     
    
    file_path = os.path.join(location_image , 'Floorplanpredict.png')
    
    cv2.imwrite(file_path, imgA)
    
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'C:/Users/geral/Documents/SeniorProj/FloorPlans-b0febaf27c5c.json'

    # TODO(developer): Uncomment and set the following variables
    project_id = "floorplan-274512";
    location = "us-central1";
    model_id = "IOD2023202550177595392";
    prediction_client = automl.PredictionServiceClient()

    # Get the full path of the model.
    model_full_id = prediction_client.model_path(
        project_id, "us-central1", model_id
    )
    #file_path = 'C:/Users/geral/Documents/FloorPlans/set1_png/file_1.png'

    
    # Read the file.
    with open(file_path, "rb") as content_file:
        content = content_file.read()

    image = automl.types.Image(image_bytes=content)
    payload = automl.types.ExamplePayload(image=image)

    # params is additional domain-specific parameters.
    # score_threshold is used to filter the result
    # https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#predictrequest
    
    params = {"score_threshold": "0.8"}

    response = prediction_client.predict(model_full_id, payload, params)
    
    result_text = os.path.join(location_image , 'Floorplanpredict.txt')
    
        
    print(response)
    #print(type(response))
            
    count=0
       
    object_list = []
    score_list = []
    percentage_list = []
    x = []
    y = []
    
    ###Acess JSON Payload#######
    
    for result in response.payload:
        
        bounding_box = result.image_object_detection.bounding_box
        object_list.append(result.display_name)
        score_list.append(result.image_object_detection.score)
        percentage_list.append("{:.2%}".format(result.image_object_detection.score)) 
        count = count+1
        
        for vertex in bounding_box.normalized_vertices:
            
        ####OBTAIN ORIGINAL COORDINATES BY MULTIPLYING NORMALIZED VALUES BY ORIGINAL DIMENSIONS######
            org_x = int(vertex.x*width) 
            org_y = int(vertex.y*height)
            x.append(org_x)
            y.append(org_y)
             
    coords_list = list(zip(x,y))    
    #print(coords_list)
    #print(count)
    print(score_list)
    print(percentage_list)
    
#####count#################
    #print(object_list)
    c = dict(Counter(object_list))
    #print(c)
        
    for k, v in c.items():
        print("{} count:  {}".format(k, v))
  
        
    #pts = np.array(list_of_coords, np.int32)
    
   
    i = 0
    
    for p, coords in enumerate(coords_list):
        
        if p%2 == 0:
            c1 = coords 
            label = object_list[i]
            score = percentage_list[i]
            out = label +" "+ score
            i=i+1
            (a,b) = c1
            cv2.putText(imgA,out,(a,b-10), cv2.FONT_HERSHEY_COMPLEX,1.5,(0,0,255),2)
        else:
            c2 = coords
            cv2.rectangle(imgA, c1, c2, (0, 0, 255), 2)
            
     
    
  
    cv2.imwrite(os.path.join(location_image, 'predicted.png'), imgA)
      
        
        

        #for result in response.payload:
            #print("Predicted class name: {}".format(result.display_name))
            #print("Predicted class score: {}".format(result.image_object_detection.score))
            #bounding_box = result.image_object_detection.bounding_box
            #print("Normalized Vertices:")
            #f.write("Predicted classname:{}\n".format(result.display_name))
            #f.write("Predicted class score:{}\n".format(result.image_object_detection.score))
            #f.write("Normalized vertices\n") 
            #for vertex in bounding_box.normalized_vertices:
                #print("\tX: {}, Y: {}".format(vertex.x, vertex.y))
                #f.write("\tX: {}, Y: {}\n".format(vertex.x, vertex.y))
                
                #vertices = cv2.boundingRect(vertex)
                
             

    # [END automl_vision_object_detection_predict]
 



    
    


2858 6775
